In [1]:
import astropy.units as u
from zero_point import zpt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.stats import sigma_clip
import scipy.optimize as spopt
import os
from pathlib import Path
from sklearn.cluster import KMeans
from bar_distance_analytic_model import compute_bar_distance
import fitting_routines as fit_functions
import plot_functions
zpt.load_tables()

In [ ]:
#b = np.arange(-6,-2,0.5)
b = np.array([-3.00])
Al = np.arange(-15,16,1)
#Al = np.array([-10])

for i in b: # run functions for sightlines
    # find folder with csvs
    dir_name = f'Gaia_Data/b_{i:0.2f}'
    os.makedirs(dir_name, exist_ok=True)
    path = Path(dir_name)

    #create folders and lists for plots
    plot_path = f'Plots/b__test_v5_{i:0.2f}'
    os.makedirs(plot_path, exist_ok=True)
    plotpath = Path(plot_path)
    ZpPara = []
    ZpParaerr = []
    PreZpPara = []
    PreZpParaerr = []
    L = [] #separate longitude list from Al in case a longitude has an error and cannot be fitted; 
           #using the Al list will cause an error for the final plot in that case
    for l in Al:
        try:
            df = pd.read_csv(path / f'l_{l:0.2f}_b_{i:0.2f}.csv') # read in csv
            redclump = fit_functions.redclumpfinder_v1(df) # find the red clump
            zero_point = fit_functions.zeropoint(redclump); # zero point corrections for Gaia
            parallax = fit_functions.finalfit(redclump, zero_point) # sigma clip parallax both before and after applying zero point correction
            ZpPara.append(parallax["zp parallax peak"])
            ZpParaerr.append(parallax["zp parallax error"])
            PreZpPara.append(parallax["prezp parallax peak"])
            PreZpParaerr.append(parallax["prezp parallax error"])
            plot_functions.RedClumpPlot(redclump, parallax, df, zero_point, plotpath=plotpath, l=l, b=i)
        except RuntimeError: # sometimes it will fail to fit and will crash
            try:
                redclump = fit_functions.redclumpfinder_v2(df) # see if it will fit if we change the spread for the first cut
                zero_point = fit_functions.zeropoint(redclump);
                parallax = fit_functions.finalfit(redclump, zero_point)
                ZpPara.append(parallax["zp parallax peak"])
                ZpParaerr.append(parallax["zp parallax error"])
                PreZpPara.append(parallax["prezp parallax peak"])
                PreZpParaerr.append(parallax["prezp parallax error"])
                plot_functions.RedClumpPlot(redclump, parallax, df, zero_point, plotpath=plotpath, l=l, b=i)
            except RuntimeError:
                print("Could not fit", f'l_{l:0.2f}_b_{i:0.2f}') # if not, save the fitting cut and 2D histogram for troubleshooting
                plot_functions.RedClumpPlot_runtime(redclump, df, plotpath=plotpath, l=l, b=i)
                continue
            continue
        except KeyError: # usually means it could find a fit, but for some reason could not go beyond plotting the histograms
            print("Key Error for", f'l_{l:0.2f}_b_{i:0.2f}')
            plot_functions.RedClumpPlot_break(redclump, df, plotpath=plotpath, l=l, b=i)
            continue
        except ValueError:
            print("Value Error for", f'l_{l:0.2f}_b_{i:0.2f}')
            plot_functions.RedClumpPlot_runtime(redclump, df, plotpath=plotpath, l=l, b=i)
            continue
        except FileNotFoundError:
            print("Could not find file", f'l_{l:0.2f}_b_{i:0.2f}')
            continue
        L.append(l)
    # save mean parallax from each sightline in a dictionary and convert to a dataframe to be used for mean parallax vs. longitude plot
    d = {"Long": L, "Mean Parallax w/ Zero Point": ZpPara, "Mean Parallax Error w/ Zero Point": ZpParaerr,
         "Mean Parallax w/o Zero Point": PreZpPara, "Mean Parallax Error w/o Zero Point": PreZpParaerr}
    meanplot = pd.DataFrame(data=d)
    meanplot.to_csv(path / f"b_{i}_final_cuts.csv") # save all cuts along a latitude to a csv
    plot_functions.meanplot(meanplot, Al, b=i, plotpath=plotpath)

c:\Users\Caoimhin\Documents\Gaia_Bar_Angle\zero_point\zpt.py:243: UserWarning: The pseudocolour of some of the 6p source(s) is outside the expected range (1.24-1.72 mag).
                 The maximum corrections are reached already at 1.24 and 1.72
  warnings.warn(
c:\ProgramData\miniforge3\Lib\site-packages\scipy\optimize\_minpack_py.py:989: RuntimeWarning: divide by zero encountered in divide
  transform = 1.0 / sigma
c:\Users\Caoimhin\Documents\Gaia_Bar_Angle\fit_functions_copy2.py:176: OptimizeWarning: Covariance of the parameters could not be estimated
  zppar,zpcov = spopt.curve_fit(zpcut,zpbc,zphist,p0=zpguess,sigma=np.sqrt(zphist))
c:\Users\Caoimhin\Documents\Gaia_Bar_Angle\fit_functions_copy2.py:181: OptimizeWarning: Covariance of the parameters could not be estimated
  ppar,pcov = spopt.curve_fit(pcut,pbc,phist,p0=pguess,sigma=np.sqrt(phist))
c:\Users\Caoimhin\Documents\Gaia_Bar_Angle\zero_point\zpt.py:243: UserWarning: The pseudocolour of some of the 6p source(s) is outside 

In [5]:
parallax["post-zp dataframe"]

,parallax,parallax error
94,0.228012,0.051357
125,0.219771,0.053137
154,0.156713,0.064010
193,0.182051,0.062447
214,0.155991,0.061904
...,...,...
31028,0.141473,0.075084
31073,0.104071,0.067113
31128,0.291052,0.069840
31152,0.022530,0.080830
